In [8]:
import os
import pickle

import pandas as pd

import mlflow
import uuid

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [9]:
from sklearn.pipeline import make_pipeline

In [10]:

#mlflow.set_tracking_uri("http://127.0.0.1:5000")
#mlflow.set_experiment("green-taxi-duration")

In [19]:
year=2021
month=2
taxi_type='green'

input_file =f'https://d37ci6vzurychx.cloudfront.net/trip-data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet'
output_file=f'output/{taxi_type}/{year:04d}-{month:02d}.parquet'

In [20]:
MLFLOW_TRACKING_URI ='http://127.0.0.1:5000'
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

#RUN_ID = os.getenv('RUN_ID')
RUN_ID ='6b736fe6c66e495e88a45b58a64eee67'
#logged_model = f's3://mlflow-models-alexey/1/{RUN_ID}/artifacts/model'



In [21]:
def generate_uuids(n):
    ride_ids=[]
    for i in range(n):
        ride_ids.append(str(uuid.uuid4()))
    return ride_ids


def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    df['ride_id'] = generate_uuids(len(df))
    return df


def prepare_dictionaries(df: pd.DataFrame):
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
   
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [22]:
def load_model(RUN_ID):
    logged_model = f'runs:/{RUN_ID}/model'
    model = mlflow.pyfunc.load_model(logged_model)
    return model


def apply_model(input_file, run_id, output_file):
    df = read_dataframe(input_file)
    dicts = prepare_dictionaries(df)

    model = load_model(run_id)
    y_pred = model.predict(dicts)

    df_result = pd.DataFrame()
    df_result['ride_id'] = df['ride_id']#there is no uniqueID in the df, can create a userid by combining unique columns, or use uuid
    df_result['lpep_pickup_datetime'] = df['lpep_pickup_datetime']
    df_result['PUlocationID']= df['PULocationID']
    df_result['DO_locationID']=df['DOLocationID']
    df_result['actual_duration']=df['duration']
    df_result['predicted_duration'] = y_pred
    df_result['diff'] =df_result['actual_duration']-df_result['predicted_duration']
    df_result['model_version'] =RUN_ID

    df_result.to_parquet(output_file, index=False)

In [ ]:
!mkdir output

In [23]:
apply_model(input_file=input_file, run_id = RUN_ID, output_file=output_file)

In [24]:
!ls output/green

2021-02.parquet 2021-03.parquet
